In [1]:
import numpy as np
from random import seed
from random import sample
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from skimage.measure import regionprops_table
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
from skimage.measure import regionprops_table

In [25]:
nucleiPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei'
clusterPath = '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10'

In [26]:
clusterFolders = ["/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/5-60/aorta04", "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/5-60/aorta05", "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/5-60/aorta06"]
clusterFolders2 = ["/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/5-60/aorta07", "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/5-60/aorta08"]

In [26]:
clusterFolders = [x[0] for x in os.walk(clusterPath)]
clusterFolders = sorted(clusterFolders)

In [27]:
clusterFolders

['/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta01',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta02',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta03',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta04',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta05',
 '/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-10/aorta06']

In [28]:
def fullNucImage(nucPath):
    scalingFactor = 10000
    for i in range(4):
        #build row block
        nuc1 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +1) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc2 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +2) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc3 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +3) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc4 =np.load(join(nucPath, "Cropped_IMG-" + str(4*i +4) + '_seg.npy'), allow_pickle=True).item()['masks']
        nuc1 = nuc1 + scalingFactor *(4*i +1)
        nuc1[nuc1 == (scalingFactor *(4*i +1))] = 0
        nuc2 = nuc2 + scalingFactor *(4*i +2)
        nuc2[nuc2 == (scalingFactor *(4*i +2))] = 0
        nuc3 = nuc3 + scalingFactor *(4*i +3)
        nuc3[nuc3 == (scalingFactor *(4*i +3))] = 0
        nuc4 = nuc4 + scalingFactor *(4*i +4)
        nuc4[nuc4 == (scalingFactor *(4*i +4))] = 0

        row = np.concatenate((nuc1, nuc2, nuc3, nuc4), axis = 1)
        if i == 0:
            nucImage = row
        else:
            nucImage = np.concatenate((nucImage, row), axis = 0)
    return nucImage

In [29]:
def clusterSize(masks, nucImage):
    accuracy_value = 0.8
    clusters = list()
    dic = {}
    dicNot = list()
    dividedOnly = masks.copy()
    marked = masks.copy()
    for mask in np.unique(masks):
        if mask != 0:
            cluster = list()
            coord = np.where(masks == mask)
            segmentationNuc = np.unique(nucImage[coord])
            for j in segmentationNuc:
                if j != 0:
                    #occurence of nucleus pixel in
                    nucOccur = np.bincount(nucImage[coord])[j]
                    #How many pixels does this nucleus have:
                    coordLen = len(np.where(nucImage == j)[0])
                    #check if most of nuclei values are in segmentation
                    if(nucOccur >= accuracy_value*coordLen):
                        cluster.append(j)
            if cluster:
                dic[mask] = cluster
                clusters.append(cluster)
            else:
                dicNot.append(mask)
                dividedOnly = np.where(dividedOnly == mask, 0, dividedOnly)
            if len(cluster) == 1:
                dividedOnly = np.where(dividedOnly == mask, 0, dividedOnly)

    clusterSize = []
    for key in dic:
        clusterSize.append(len(dic[key]))
    return clusterSize, dicNot, marked, dividedOnly

In [30]:
def dataFrameHelp(age, aorta, values):
    df_help = pd.DataFrame(columns= ['age', 'aorta', 'value'])
    df_help['value'] = values
    df_help['age'] = age
    df_help['aorta'] = aorta
    return df_help

In [31]:
def kNN(array, neighbor):
    nbrs = NearestNeighbors(n_neighbors=neighbor, algorithm='ball_tree').fit(array)
    distances, indices = nbrs.kneighbors(array)
    return nbrs, distances, indices

In [32]:
def randomkNNDistances(nucIDs,len, df):
    selectedRandom = sample(nucIDs, len)
    dfRandom = df[df['label'].isin(selectedRandom)]
    nucCoordRandom = dfRandom[['centroid-0', 'centroid-1']].to_numpy()
    nbrsRandom = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(nucCoordRandom)
    distancesRand, indicesRandom = nbrsRandom.kneighbors(nucCoordRandom)
    return distancesRand

In [33]:
def randomkNNCluster(nucIDs,lens, df, simulationNumber):
    seed(42)
    markedRandom = []
    dividedRandom = []
    for _ in range(simulationNumber):
        if lens[0] > 7:
            markedRandom.append(np.asarray(randomkNNDistances(nucIDs,lens[0], df)))
        if lens[1] > 7:
            dividedRandom.append(np.asarray(randomkNNDistances(nucIDs, lens[1], df)))
    return np.asarray(markedRandom) , np.asarray(dividedRandom)

In [34]:
dfMarked1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfMarked4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

dfDivided1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfDivided4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

In [34]:
dfMarkedResult = pd.DataFrame()

In [35]:
for folder in clusterFolders:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        print(age)
        print(aorta)
        nucPath = join(nucleiPath, age, aorta)
        nucImage = fullNucImage(nucPath)
        masks = np.load(join(folder, file), allow_pickle=True).item()['masks']

        clusterSizes, dicNot, marked, dividedOnly = clusterSize(masks, nucImage)

        unique, counts = np.unique(clusterSizes, return_counts=True)
        numberMarked = counts.sum() + len(dicNot)
        numberDivided = counts[1:].sum()
        print(len(np.unique(marked)))
        print(numberMarked)

        prop_Marked = regionprops_table(marked, properties = ('label', 'centroid'))
        dfMarked = pd.DataFrame(prop_Marked)

        nbrs, distancesMarked, indices = kNN(dfMarked.iloc[:,1:],6)
        dfHelp = pd.DataFrame(distancesMarked)
        dfHelp["age"] = age
        dfHelp["aorta"] = aorta
        dfMarkedResult = pd.concat([dfMarkedResult, dfHelp])

        #prop_Divided = regionprops_table(dividedOnly, properties = ('label', 'centroid'))
        #dfDivided = pd.DataFrame(prop_Divided)
        #nbrs, distancesDivided, indices = kNN(dfDivided.iloc[:,1:],6)

        #prop_nuc = regionprops_table(nucImage, properties = ('label', 'centroid'))
        #dfNuc = pd.DataFrame(prop_nuc)
        #nucIDs = dfNuc['label'].tolist()
        #lens = [numberMarked, numberDivided]
        #markedRandom, dividedRandom =  randomkNNCluster(nucIDs,lens, dfNuc, 1000)

        age = nucPath.split("/")[-2]
        aorta = nucPath.split("/")[-1]

        #dfMarked1NN = dfMarked1NN.append(dataFrameHelp(age, aorta, distancesMarked[:,1]/np.mean(markedRandom[:,:,1].flatten()) -1 ))
        #dfMarked2NN = dfMarked2NN.append(dataFrameHelp(age, aorta, distancesMarked[:,2]/np.mean(markedRandom[:,:,2].flatten()) -1 ))
        #dfMarked3NN = dfMarked3NN.append(dataFrameHelp(age, aorta, distancesMarked[:,3]/np.mean(markedRandom[:,:,3].flatten()) -1 ))
        #dfMarked4NN = dfMarked4NN.append(dataFrameHelp(age, aorta, distancesMarked[:,4]/np.mean(markedRandom[:,:,4].flatten()) -1 ))
        #dfMarked5NN = dfMarked5NN.append(dataFrameHelp(age, aorta, distancesMarked[:,5]/np.mean(markedRandom[:,:,5].flatten()) -1 ))

        #dfDivided1NN = dfDivided1NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        #dfDivided2NN = dfDivided2NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        #dfDivided3NN = dfDivided3NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        #dfDivided4NN = dfDivided4NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        #dfDivided5NN = dfDivided5NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))

0-10
aorta01
634
633
0-10
aorta02
563
562
0-10
aorta03
426
425
0-10
aorta04
348
347
0-10
aorta05
412
411
0-10
aorta06
279
278


In [36]:
dfMarkedResult

,0,1,2,3,4,5,age,aorta
0,0.0,45.071287,61.416105,65.382754,65.419472,68.097205,0-10,aorta01
1,0.0,50.160017,57.688693,60.235186,60.606848,62.787766,0-10,aorta01
2,0.0,41.433286,49.582140,51.324984,55.377924,74.504462,0-10,aorta01
3,0.0,26.402508,44.747515,46.714342,57.974775,61.101135,0-10,aorta01
4,0.0,17.882815,32.898724,39.322564,47.061509,65.420026,0-10,aorta01
...,...,...,...,...,...,...,...,...
273,0.0,19.333887,100.354670,148.374319,170.702293,184.603020,0-10,aorta06
274,0.0,56.953549,58.417655,61.472491,62.569630,93.500582,0-10,aorta06
275,0.0,33.117730,33.440292,50.191174,65.633712,68.776118,0-10,aorta06
276,0.0,36.546462,49.248244,53.232130,70.126903,71.989638,0-10,aorta06


In [24]:
savePath = "/Users/jones/Downloads"
dfMarkedResult.to_csv(join(savePath, "markedkNN_0-10.csv"), index=False)

In [36]:
dfMarked1NN['kNNType'] = "1NN"
dfMarked2NN['kNNType'] = "2NN"
dfMarked3NN['kNNType'] = "3NN"
dfMarked4NN['kNNType'] = "4NN"
dfMarked5NN['kNNType'] = "5NN"
dfMarkedAll = pd.concat([dfMarked1NN, dfMarked2NN, dfMarked3NN, dfMarked4NN, dfMarked5NN])

In [37]:
dfDivided1NN['kNNType'] = "1NN"
dfDivided2NN['kNNType'] = "2NN"
dfDivided3NN['kNNType'] = "3NN"
dfDivided4NN['kNNType'] = "4NN"
dfDivided5NN['kNNType'] = "5NN"
dfDividedAll = pd.concat([dfDivided1NN, dfDivided2NN, dfDivided3NN, dfDivided4NN, dfDivided5NN])

In [38]:
savePath = "/Users/jones/Downloads"
dfMarkedAll.to_csv(join(savePath, "markedClustersRedkNN_5-60_1.csv"), index=False)
dfDividedAll.to_csv(join(savePath, "dividedClustersRedkNN_5-60_1.csv"), index=False)

In [41]:
def clusterSize5_60_A78(masks, nucImage):
    accuracy_value = 0.8
    clusters = list()
    dic = {}
    dicNot = list()
    dividedOnly = masks.copy()
    marked = masks.copy()
    for mask in np.unique(masks):
        if mask != 0:
            cluster = list()
            coord = np.where(masks == mask)
            segmentationNuc = np.unique(nucImage[coord])
            for j in segmentationNuc:
                if j != 0:
                    #occurence of nucleus pixel in
                    nucOccur = np.bincount(nucImage[coord])[j]
                    #How many pixels does this nucleus have:
                    coordLen = len(np.where(nucImage == j)[0])
                    #check if most of nuclei values are in segmentation
                    if(nucOccur >= accuracy_value*coordLen):
                        cluster.append(j)
            if cluster:
                dic[mask] = cluster
                clusters.append(cluster)
            else:
                dicNot.append(mask)
                dividedOnly = np.where(dividedOnly == mask, 0, dividedOnly)
                marked = np.where(marked == mask, 0, marked)
            if len(cluster) == 1:
                dividedOnly = np.where(dividedOnly == mask, 0, dividedOnly)

    clusterSize = []
    for key in dic:
        clusterSize.append(len(dic[key]))
    return clusterSize, dicNot, marked, dividedOnly

In [42]:
dfMarked1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfMarked4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfMarked5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

dfDivided1NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided2NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided3NN = pd.DataFrame(columns= ['age', 'aorta','value'])
dfDivided4NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])
dfDivided5NN = pd.DataFrame(columns= ['age', 'aorta', 'value'])

for folder in clusterFolders2:
    os.chdir(folder)
    files = [f for f in listdir(folder) if isfile(join(folder, f)) if ".DS_Store" not in f if ".npy" in f if "._" not in f]
    for file in files:
        aorta = folder.split("/")[-1]
        age = folder.split("/")[-2]
        print(age)
        print(aorta)
        nucPath = join(nucleiPath, age, aorta)
        nucImage = fullNucImage(nucPath)
        masks = np.load(join(folder, file), allow_pickle=True).item()['masks']

        clusterSizes, dicNot, marked, dividedOnly = clusterSize5_60_A78(masks, nucImage)

        unique, counts = np.unique(clusterSizes, return_counts=True)
        numberMarked = counts.sum()
        numberDivided = counts[1:].sum()
        print(len(np.unique(marked)))
        print(numberMarked)

        prop_Marked = regionprops_table(marked, properties = ('label', 'centroid'))
        dfMarked = pd.DataFrame(prop_Marked)

        nbrs, distancesMarked, indices = kNN(dfMarked.iloc[:,1:],6)

        prop_Divided = regionprops_table(dividedOnly, properties = ('label', 'centroid'))
        dfDivided = pd.DataFrame(prop_Divided)
        nbrs, distancesDivided, indices = kNN(dfDivided.iloc[:,1:],6)

        prop_nuc = regionprops_table(nucImage, properties = ('label', 'centroid'))
        dfNuc = pd.DataFrame(prop_nuc)
        nucIDs = dfNuc['label'].tolist()
        lens = [numberMarked, numberDivided]
        markedRandom, dividedRandom =  randomkNNCluster(nucIDs,lens, dfNuc, 1000)

        age = nucPath.split("/")[-2]
        aorta = nucPath.split("/")[-1]

        dfMarked1NN = dfMarked1NN.append(dataFrameHelp(age, aorta, distancesMarked[:,1]/np.mean(markedRandom[:,:,1].flatten()) -1 ))
        dfMarked2NN = dfMarked2NN.append(dataFrameHelp(age, aorta, distancesMarked[:,2]/np.mean(markedRandom[:,:,2].flatten()) -1 ))
        dfMarked3NN = dfMarked3NN.append(dataFrameHelp(age, aorta, distancesMarked[:,3]/np.mean(markedRandom[:,:,3].flatten()) -1 ))
        dfMarked4NN = dfMarked4NN.append(dataFrameHelp(age, aorta, distancesMarked[:,4]/np.mean(markedRandom[:,:,4].flatten()) -1 ))
        dfMarked5NN = dfMarked5NN.append(dataFrameHelp(age, aorta, distancesMarked[:,5]/np.mean(markedRandom[:,:,5].flatten()) -1 ))

        dfDivided1NN = dfDivided1NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        dfDivided2NN = dfDivided2NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        dfDivided3NN = dfDivided3NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        dfDivided4NN = dfDivided4NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))
        dfDivided5NN = dfDivided5NN.append(dataFrameHelp(age, aorta, distancesDivided[:,1]/np.mean(dividedRandom[:,:,1].flatten()) -1 ))

5-60
aorta07
248
247


/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked1NN = dfMarked1NN.append(dataFrameHelp(age, aorta, distancesMarked[:,1]/np.mean(markedRandom[:,:,1].flatten()) -1 ))
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked2NN = dfMarked2NN.append(dataFrameHelp(age, aorta, distancesMarked[:,2]/np.mean(markedRandom[:,:,2].flatten()) -1 ))
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked3NN = dfMarked3NN.append(dataFrameHelp(age, aorta, distancesMarked[:,3]/np.mean(markedRandom[:,:,3].

5-60
aorta08
417
416


/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked1NN = dfMarked1NN.append(dataFrameHelp(age, aorta, distancesMarked[:,1]/np.mean(markedRandom[:,:,1].flatten()) -1 ))
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked2NN = dfMarked2NN.append(dataFrameHelp(age, aorta, distancesMarked[:,2]/np.mean(markedRandom[:,:,2].flatten()) -1 ))
/var/folders/6r/4p5g0w_j4jv11sr24nv1b0140000gn/T/ipykernel_35215/252534.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMarked3NN = dfMarked3NN.append(dataFrameHelp(age, aorta, distancesMarked[:,3]/np.mean(markedRandom[:,:,3].

In [43]:
dfMarked1NN['kNNType'] = "1NN"
dfMarked2NN['kNNType'] = "2NN"
dfMarked3NN['kNNType'] = "3NN"
dfMarked4NN['kNNType'] = "4NN"
dfMarked5NN['kNNType'] = "5NN"
dfMarkedAll = pd.concat([dfMarked1NN, dfMarked2NN, dfMarked3NN, dfMarked4NN, dfMarked5NN])

dfDivided1NN['kNNType'] = "1NN"
dfDivided2NN['kNNType'] = "2NN"
dfDivided3NN['kNNType'] = "3NN"
dfDivided4NN['kNNType'] = "4NN"
dfDivided5NN['kNNType'] = "5NN"
dfDividedAll = pd.concat([dfDivided1NN, dfDivided2NN, dfDivided3NN, dfDivided4NN, dfDivided5NN])

In [44]:
savePath = "/Users/jones/Downloads"
dfMarkedAll.to_csv(join(savePath, "markedClustersRedkNN_5-60_2.csv"), index=False)
dfDividedAll.to_csv(join(savePath, "dividedClustersRedkNN_5-60_2.csv"), index=False)

In [45]:
#combine dfs
dfMarked1 = pd.read_csv("/Users/jones/Downloads/markedClustersRedkNN_3.csv")
dfMarked2 = pd.read_csv("/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/markedClustersRedkNN_0-5.csv")
dfMarked3 = pd.read_csv("/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/markedClustersRedkNN_0-10.csv")
dfMarked4 = pd.read_csv("/Users/jones/Downloads/markedClustersRedkNN_5-60_1.csv")
dfMarked5 = pd.read_csv("/Users/jones/Downloads/markedClustersRedkNN_5-60_2.csv")

dfMarkedAllNow = pd.concat([dfMarked1, dfMarked2, dfMarked3, dfMarked4, dfMarked5])

In [46]:
dfMarkedAllNow

,age,aorta,value,kNNType
0,10-21,aorta01,0.538041,1NN
1,10-21,aorta01,-0.102917,1NN
2,10-21,aorta01,0.054030,1NN
3,10-21,aorta01,-0.065480,1NN
4,10-21,aorta01,0.943803,1NN
...,...,...,...,...
3310,5-60,aorta08,0.217698,5NN
3311,5-60,aorta08,-0.146573,5NN
3312,5-60,aorta08,0.002059,5NN
3313,5-60,aorta08,0.099470,5NN


In [47]:
dfMarkedAllNow.to_csv(join(savePath, "markedClustersRedkNN_ALL_v3.csv"), index=False)

In [48]:
#combine dfs
dfDivided1 = pd.read_csv("/Users/jones/Downloads/dividedClustersRedkNN_3.csv")
dfDivided2 = pd.read_csv("/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/dividedClustersRedkNN_0-5.csv")
dfDivided3 = pd.read_csv("/Volumes/Aortas/Aorta/results/inducedRainbow/spatialPattern/dividedClustersRedkNN_0-10.csv")
dfDivided4 = pd.read_csv("/Users/jones/Downloads/dividedClustersRedkNN_5-60_1.csv")
dfDivided5 = pd.read_csv("/Users/jones/Downloads/dividedClustersRedkNN_5-60_2.csv")

dfDividedAllNow = pd.concat([dfDivided1, dfDivided2, dfDivided3, dfDivided4, dfDivided5])

In [70]:
dfDividedAllNow.to_csv(join(savePath, "dividedClustersRedkNN_ALL_v3.csv"), index=False)

In [51]:
dfDividedAllNow

,age,aorta,value,kNNType
0,10-21,aorta01,0.082806,1NN
1,10-21,aorta01,0.082806,1NN
2,10-21,aorta01,0.145923,1NN
3,10-21,aorta01,0.751155,1NN
4,10-21,aorta01,-0.099933,1NN
...,...,...,...,...
1445,5-60,aorta08,0.818402,5NN
1446,5-60,aorta08,0.570465,5NN
1447,5-60,aorta08,-0.110917,5NN
1448,5-60,aorta08,-0.058467,5NN


In [50]:
dfDividedAllNow.groupby(["age", "aorta"]).count()

value  kNNType
age   aorta                  
0-10  aorta01   1665     1665
      aorta02   1495     1495
      aorta03    890      890
      aorta04    640      640
      aorta05    775      775
      aorta06    530      530
0-5   aorta01    575      575
      aorta02    470      470
      aorta03    600      600
      aorta04    850      850
      aorta05    545      545
      aorta06    565      565
10-21 aorta01    785      785
      aorta02    305      305
      aorta03    615      615
      aorta04    385      385
      aorta05    690      690
10-30 aorta01    990      990
      aorta02   1220     1220
      aorta03    575      575
      aorta04    715      715
      aorta05    635      635
      aorta06    800      800
      aorta07    865      865
10-60 aorta01    540      540
      aorta02    930      930
      aorta03   1040     1040
      aorta04   1180     1180
      aorta05    455      455
      aorta06   1265     1265
21-30 aorta01    135      135
      aorta02     80       80
      aorta03    100      100
30-60 aorta01    275      275
      aorta02    125      125
      aorta03    225      225
      aorta04    230      230
      aorta05     80       80
5-10  aorta01   1180     1180
      aorta02    600      600
      aorta03    445      445
      aorta04    945      945
      aorta06    480      480
      aorta07    665      665
      aorta08    680      680
5-30  aorta01   1215     1215
      aorta02    940      940
      aorta03    560      560
      aorta04    915      915
      aorta05    650      650
      aorta06   1140     1140
5-60  aorta01    685      685
      aorta02   1275     1275
      aorta03    675      675
      aorta04   1490     1490
      aorta05   1165     1165
      aorta06   1425     1425
      aorta07    500      500
      aorta08    950      950

In [53]:
dfDividedAllNow

,age,aorta,value,kNNType
0,10-21,aorta01,0.082806,1NN
1,10-21,aorta01,0.082806,1NN
2,10-21,aorta01,0.145923,1NN
3,10-21,aorta01,0.751155,1NN
4,10-21,aorta01,-0.099933,1NN
...,...,...,...,...
1445,5-60,aorta08,0.818402,5NN
1446,5-60,aorta08,0.570465,5NN
1447,5-60,aorta08,-0.110917,5NN
1448,5-60,aorta08,-0.058467,5NN


In [59]:
dfDividedAllNow = dfDividedAllNow.reset_index(drop=True)

In [68]:
dfDividedAllNow = dfDividedAllNow.drop(index=[3284,8942,14600,20258,25916]).reset_index(drop=True)

In [69]:
dfDividedAllNow

,age,aorta,value,kNNType
0,10-21,aorta01,0.082806,1NN
1,10-21,aorta01,0.082806,1NN
2,10-21,aorta01,0.145923,1NN
3,10-21,aorta01,0.751155,1NN
4,10-21,aorta01,-0.099933,1NN
...,...,...,...,...
43410,5-60,aorta08,0.818402,5NN
43411,5-60,aorta08,0.570465,5NN
43412,5-60,aorta08,-0.110917,5NN
43413,5-60,aorta08,-0.058467,5NN


In [106]:
testPath = "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/red/0-5/aorta01/cellposeSegm_RedP0-5_aorta01_50Thre_finalNew_seg.npy"
test = np.load(testPath, allow_pickle=True).item()['masks']
nucPath = "/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei/0-5/aorta01"

In [9]:
nucImage =  fullNucImage("/Volumes/Aortas/Aorta/segmentations/inducedRainbow/final/nuclei/0-5/aorta01")

In [59]:
clusterSizes, dicNot, marked, dividedOnly = clusterSizes(test, nucImage)

In [60]:
unique, counts = np.unique(clusterSizes, return_counts=True)

In [61]:
numberMarked = counts.sum() + len(dicNot)
numberDivided = counts[1:].sum()

In [62]:
numberMarked

524

In [63]:
len(np.unique(dividedOnly))

116

In [68]:
prop_dict = regionprops_table(marked, properties = ('label', 'centroid'))
df_help = pd.DataFrame(prop_dict)

In [69]:
df_help

,label,centroid-0,centroid-1
0,11,11.282759,602.696552
1,12,18.978495,764.903226
2,14,25.281106,710.414747
3,16,26.205882,810.470588
4,17,22.235294,732.176471
...,...,...,...
519,918,1082.463415,840.473171
520,919,933.735395,829.109966
521,920,962.668394,825.238342
522,921,466.717277,750.785340


In [81]:
nbrs, distances, indices = kNN(df_help.iloc[:,1:],6)

In [82]:
distances

array([[ 0.        , 27.07602602, 38.64095015, 55.21295917, 57.00141225,
        59.18617971],
       [ 0.        , 32.88840601, 38.11990579, 43.07848539, 46.13696622,
        48.76343817],
       [ 0.        , 21.97383907, 34.79342903, 54.85177556, 72.78755393,
        77.23105475],
       ...,
       [ 0.        , 29.19088689, 31.57916987, 40.41094645, 40.74168706,
        47.795374  ],
       [ 0.        , 12.75242949, 17.56221096, 26.49912445, 30.81518012,
        46.27179345],
       [ 0.        , 26.288571  , 26.49912445, 32.09216127, 32.69117356,
        34.13697575]])

In [85]:
prop_nuc = regionprops_table(nucImage, properties = ('label', 'centroid'))
dfNuc = pd.DataFrame(prop_nuc)
nucIDs = dfNuc['label'].tolist()

In [88]:
lens = [numberMarked, numberDivided]
markedRandom, dividedRandom =  randomkNNCluster(nucIDs,lens, dfNuc, 1000)

In [103]:
distances[:,5]/(np.mean(markedRandom[:,:,5].flatten())) -1

array([ 1.13371873e-02, -1.66760236e-01,  3.19676959e-01, -1.34089302e-01,
        3.55342074e-02,  3.67551412e-01,  6.99998152e-02,  1.48735483e-01,
       -4.90623955e-02, -3.38015704e-01, -8.11036834e-02,  6.18061453e-02,
       -2.08671586e-01, -1.13073632e-01,  6.92814555e-02, -2.50457871e-01,
       -3.31706072e-01, -1.66760236e-01, -3.11044831e-02, -2.93880724e-01,
       -2.51307042e-01, -8.69034404e-02, -2.94595195e-02,  9.11123040e-02,
       -2.70684428e-01,  2.47943324e-01, -3.31706072e-01, -1.08784440e-01,
        1.25738460e-02, -4.37331433e-02, -3.55203143e-01, -2.61183145e-01,
        1.78130574e-01, -1.42790553e-01, -6.83479208e-02, -1.92683977e-01,
       -1.42790553e-01,  8.15371662e-02,  1.75979576e-02,  1.14121971e-01,
        2.91563984e-01, -2.92295467e-01, -1.02792009e-01,  4.79348871e-02,
        2.61658386e-01, -1.48594921e-01,  1.75353982e-01,  1.81968086e-01,
        7.15158767e-02, -1.62388767e-01,  6.86492890e-02, -2.83035623e-01,
        6.45674185e-02,  

In [91]:
markedRandom[0]

array([[ 0.        , 12.57026172, 28.99427622, 39.20935268, 51.37525051,
        59.40196081],
       [ 0.        , 12.57026172, 25.17282016, 47.60136667, 48.70116634,
        49.75317313],
       [ 0.        , 28.09990988, 37.14072569, 37.84169871, 39.20935268,
        47.60136667],
       ...,
       [ 0.        , 21.43994843, 49.65600843, 55.71730045, 57.59672404,
        70.19662121],
       [ 0.        , 21.43994843, 30.426473  , 36.20560879, 38.2190552 ,
        50.45459182],
       [ 0.        , 17.71642057, 21.89951268, 24.82980346, 36.20560879,
        57.59672404]])